In [ ]:
import json
import datetime as dt

In [ ]:
import graphlab

# Lets explore the tweets retrieved

In [ ]:
Tweets = graphlab.SFrame.read_json('tweets2.JSON')
graphlab.canvas.set_target('ipynb')

## Decide what data is relevant to Firebird 

In [ ]:
Tweets['date_time'] = Tweets['created_at'].apply(lambda x: dt.datetime.strptime(x, '%a %b %d %H:%M:%S +0000 %Y'))

In [ ]:
Tweets['date_time'][0]

In [ ]:
graphlab.canvas.set_target('ipynb')

In [ ]:
Tweets.sort('retweet_count', ascending=False)

In [ ]:
Tweets['word_count'] = graphlab.text_analytics.count_words(Tweets['text'], to_lower=True)

In [ ]:
Tweets['bigram_count'] = graphlab.text_analytics.count_ngrams(Tweets['text'], n=2, to_lower=True, method='word', ignore_punct=True)

In [ ]:
Tweets['trigram_count'] = graphlab.text_analytics.count_ngrams(Tweets['text'], n=3, to_lower=True, method='word', ignore_punct=True)

In [ ]:
Tweets['#_of_hashtags'] = Tweets.apply(lambda x: len(x['entities']['hashtags']))

In [ ]:
Tweets['hashtags'] = Tweets.apply(lambda x: x['entities']['hashtags'])

In [ ]:
Tweets['#_of_links'] = Tweets.apply(lambda x: len(x['entities']['urls']))

In [ ]:
Tweets['words_in_text'] = Tweets.apply(lambda x: len(x['word_count']))

### Get rid of Users with less than 100 followers and only unique Tweets

In [ ]:
len(Tweets)

In [ ]:
Tweets = Tweets[Tweets['followed_by'] > 100]

In [ ]:
len(Tweets)

# Time to create the prediction model

## Create Training and test datasets

In [ ]:
train_data,test_data = Tweets.random_split(.8, seed=0)

In [ ]:
all_features = ['word_count',
          'bigram_count',
          'trigram_count',
          'friended_by',
          'followed_by',
          '#_of_lists',
          '#_of_hashtags',
          '#_of_links',
          'words_in_text']
gram_features = [ 'word_count', 'trigram_count', 'bigram_count' ]

In [ ]:
tweet_ngram_model = graphlab.linear_regression.create(train_data,
                                                target='retweet_count',
                                                features=gram_features,
                                                validation_set=None,
                                                l1_penalty=.5,
                                                max_iterations=25)

In [ ]:
coEF = tweet_ngram_model.get('coefficients')

In [ ]:
coEf = coEF.sort('value', ascending=False)

In [ ]:
coEf[coEf['name'] == 'word_count'].print_rows(50)

In [ ]:
coEf[coEf['name'] == 'bigram_count']

In [ ]:
coEf[coEf['name'] == 'trigram_count'].print_rows(30)